In [14]:
import scanpy as sc
import scvi
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [72]:
def pp(sample_path):
    # Load the sample data from .h5 file
    adata = sc.read_10x_h5(sample_path)
    adata.var_names_make_unique()

    # Continue with your analysis steps as before
    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train(batch_size = 129) #change the code vae.train() to vae.train(batch_size = 129)
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df['prediction'] = solo.predict(soft=False)
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]

    adata.obs['Sample'] = os.path.basename(sample_path)  # Extract the filename as the sample name

    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]

    sc.pp.filter_cells(adata, min_genes=200)
    adata.var['mt'] = adata.var_names.str.startswith('Mt-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    
    # You can return or do further processing with 'adata' for each sample here
    # For example, you can save it to a list or dictionary for later use.

    return adata

In [73]:
sample_names = [
    'cE14F1.h5', 'cE14F2.h5', 'cE14F3.h5', 'cE14M1.h5', 'cE14M2.h5', 'cE14M3.h5',
    'cGFPF1.h5', 'cGFPF2.h5', 'cGFPF3.h5', 'cGFPM1.h5', 'cGFPM2.h5', 'cGFPM3.h5',
     'eERMF1.h5', 'eERMF2.h5', 'eERMF3.h5', 'eERMM1.h5', 'eERMM2.h5',  'eERMM3.h5',
     'eSTRF1.h5', 'eSTRF2.h5', 'eSTRF3.h5', 'eSTRM1.h5', 'eSTRM2.h5', 'eSTRM3.h5',
     'lERMF1.h5', 'lERMF2.h5', 'lERMF3.h5', 'lERMM1.h5',  'lERMM2.h5', 'lERMM3.h5',
     'lSTRF1.h5', 'lSTRF2.h5', 'lSTRF3.h5', 'lSTRM1.h5', 'lSTRM2.h5', 'lSTRM3.h5'
]

# Base directory where the files are located
base_dir = '/home/zia/hippo_h5_files'

# Generate the list of full file paths
sample_paths = [os.path.join(base_dir, sample_name) for sample_name in sample_names]

# Create an empty list to store the processed data for each sample
processed_data_list = []

# Loop through the sample paths and process each sample
for sample_path in sample_paths:
    processed_data = pp(sample_path)
    processed_data_list.append(processed_data)

# Now, processed_data_list contains the processed data for all samples.
# You can access and analyze each processed dataset as needed.

/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/zia/.local/lib/python3.10/site-packages/anndata/_core/an

Epoch 400/400: 100%|█| 400/400 [11:07<00:00,  1.65s/it, v_num=1, train_loss_step

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█| 400/400 [11:07<00:00,  1.67s/it, v_num=1, train_loss_step
INFO     Creating doublets, preparing SOLO model.                                                                  


/home/zia/.local/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█| 400/400 [11:06<00:00,  1.67s/it, v_num=1, train_loss_step
INFO     Creating doublets, preparing SOLO model.                                                                  


/home/zia/.local/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 1/400:   0%|                                      | 0/400 [00:00<?, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 128])

In [74]:
processed_data_list

[AnnData object with n_obs × n_vars = 3833 × 2000
     obs: '_scvi_batch', '_scvi_labels', 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'hvg', '_scvi_uuid', '_scvi_manager_uuid',
 AnnData object with n_obs × n_vars = 2799 × 2000
     obs: '_scvi_batch', '_scvi_labels', 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: 'hvg', '_scvi_uuid', '_scvi_manager_uuid',
 AnnData object with n_obs × n_vars = 3534 × 2000
    

In [76]:
processed_data.obs

,_scvi_batch,_scvi_labels,Sample,doublet,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
AAACCCAAGCCTTCTC-1,0,0,lERMF3.h5,False,248,248,478.0,0.0,0.000000
AAACCCAAGTCTCCTC-1,0,0,lERMF3.h5,False,233,233,444.0,1.0,0.225225
AAACCCACAGGCATGA-1,0,0,lERMF3.h5,False,219,219,432.0,1.0,0.231481
AAACCCAGTGGGCTCT-1,0,0,lERMF3.h5,False,275,275,565.0,0.0,0.000000
AAACCCAGTTGCGGAA-1,0,0,lERMF3.h5,False,211,211,375.0,3.0,0.800000
...,...,...,...,...,...,...,...,...,...
TTTGTTGAGGTGCGAT-1,0,0,lERMF3.h5,False,250,250,496.0,0.0,0.000000
TTTGTTGCAAGTGCAG-1,0,0,lERMF3.h5,False,400,400,1341.0,0.0,0.000000
TTTGTTGTCCAGGACC-1,0,0,lERMF3.h5,False,280,280,770.0,1.0,0.129870
TTTGTTGTCGTGCGAC-1,0,0,lERMF3.h5,False,204,204,351.0,4.0,1.139601


In [77]:
processed_data.var

,gene_ids,feature_types,genome,n_cells,highly_variable,highly_variable_rank,means,variances,variances_norm,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
Sv2b,ENSRNOG00000011160,Gene Expression,Rattus_norvegicus_genome,1438,True,238.0,0.605421,2.393376,2.121372,False,1107,1.058172,61.668975,3056.0
ENSRNOG00000066257,ENSRNOG00000066257,Gene Expression,Rattus_norvegicus_genome,571,True,748.0,0.127167,0.222282,1.459024,False,432,0.211911,85.041551,612.0
ENSRNOG00000069390,ENSRNOG00000069390,Gene Expression,Rattus_norvegicus_genome,13,True,998.0,0.002364,0.003304,1.333297,False,12,0.004848,99.584488,14.0
AABR07071986.1,ENSRNOG00000031947,Gene Expression,Rattus_norvegicus_genome,120,True,1306.0,0.020485,0.028264,1.238198,False,62,0.024584,97.853186,71.0
ENSRNOG00000064423,ENSRNOG00000064423,Gene Expression,Rattus_norvegicus_genome,268,True,65.0,0.088717,0.351939,3.403594,False,204,0.153740,92.936288,444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mt-nd4l,ENSRNOG00000031053,Gene Expression,Rattus_norvegicus_genome,317,True,261.0,0.063347,0.146972,2.049089,True,186,0.082756,93.559557,239.0
Mt-nd4,ENSRNOG00000029707,Gene Expression,Rattus_norvegicus_genome,1288,True,89.0,0.359281,3.157894,3.174999,True,738,0.435249,74.445983,1257.0
Mt-nd5,ENSRNOG00000029971,Gene Expression,Rattus_norvegicus_genome,257,True,1951.0,0.045540,0.057658,1.134622,True,130,0.051939,95.498615,150.0
Mt-cyb,ENSRNOG00000031766,Gene Expression,Rattus_norvegicus_genome,1234,True,96.0,0.317208,2.025920,3.091233,True,660,0.373961,77.146814,1080.0
